In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_features = pd.read_csv('/content/drive/My Drive/Collider Detection/data/train_features.csv')
test_features = pd.read_csv('/content/drive/My Drive/Collider Detection/data/test_features.csv')
train_target = pd.read_csv('/content/drive/My Drive/Collider Detection/data/train_target.csv')
submission = pd.read_csv('/content/drive/My Drive/Collider Detection/data/sample_submission.csv', index_col = 'id')

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
TIME_COLUMNS = []  # train(2800, 375, 1), test(700, 375, 1)
for i in range(1, 376):
    TIME_COLUMNS.append(str(i)+'_time')
SENSOR1_COLUMNS = []  # train(2800, 375, 4), test(700, 375, 4)
SENSOR2_COLUMNS = []
SENSOR3_COLUMNS = []
SENSOR4_COLUMNS = []
for i in range(1, 376):
    SENSOR1_COLUMNS.append('S1_'+str(i))
    SENSOR2_COLUMNS.append('S2_'+str(i))
    SENSOR3_COLUMNS.append('S3_'+str(i))
    SENSOR4_COLUMNS.append('S4_'+str(i))

TGT_COLUMNS = ['X', 'Y', 'M', 'V']

In [ ]:
train_features = train_features.pivot(index='id', columns = 'Time', values = ['Time', 'S1', 'S2', 'S3', 'S4'])
test_features = test_features.pivot(index = 'id', columns = 'Time', values = ['Time', 'S1', 'S2', 'S3', 'S4'])

In [ ]:
train_features.columns = [TIME_COLUMNS+SENSOR1_COLUMNS+SENSOR2_COLUMNS+SENSOR3_COLUMNS+SENSOR4_COLUMNS]
test_features.columns = [TIME_COLUMNS+SENSOR1_COLUMNS+SENSOR2_COLUMNS+SENSOR3_COLUMNS+SENSOR4_COLUMNS]

In [ ]:
target_scaler = MinMaxScaler(feature_range=(-1,1)).fit(train_target[TGT_COLUMNS])
sensor1_scaler = StandardScaler().fit(train_features[SENSOR1_COLUMNS].values.ravel().reshape(-1, 1))
sensor2_scaler = StandardScaler().fit(train_features[SENSOR2_COLUMNS].values.ravel().reshape(-1, 1))
sensor3_scaler = StandardScaler().fit(train_features[SENSOR3_COLUMNS].values.ravel().reshape(-1, 1))
sensor4_scaler = StandardScaler().fit(train_features[SENSOR4_COLUMNS].values.ravel().reshape(-1, 1))
for df in [train_features, test_features]:
    for f in TIME_COLUMNS:
        df[f] = df[[f]]/0.001496
    for f in SENSOR1_COLUMNS:
        df[f] = sensor1_scaler.transform(df[[f]])
    for f in SENSOR2_COLUMNS:
        df[f] = sensor2_scaler.transform(df[[f]])
    for f in SENSOR3_COLUMNS:
        df[f] = sensor3_scaler.transform(df[[f]])
    for f in SENSOR4_COLUMNS:
        df[f] = sensor4_scaler.transform(df[[f]])

train_target[TGT_COLUMNS] = target_scaler.transform(train_target[TGT_COLUMNS])

In [ ]:
class DataBatchGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, y=None, shuffle=True, batch_size=256):
        self.batch_size = batch_size
        self.time_cols = data[TIME_COLUMNS].values.copy()
        self.sensor1_cols = data[SENSOR1_COLUMNS].values.copy()
        self.sensor2_cols = data[SENSOR2_COLUMNS].values.copy()
        self.sensor3_cols = data[SENSOR3_COLUMNS].values.copy()
        self.sensor4_cols = data[SENSOR4_COLUMNS].values.copy()
        if y is not None:
            self.targets = y[TGT_COLUMNS].values.copy()
        else:
            self.targets = None
        self.idxs = np.array(range(data.shape[0]))
        self.shuffle = shuffle

        if self.shuffle:
            np.random.shuffle(self.idxs)

    def __len__(self):
        return math.ceil(self.idxs.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        iids = self.idxs[idx * self.batch_size:idx * self.batch_size + self.batch_size]
        lids = len(iids)
        
        inputs = []
        inputs.append(self.time_cols[iids])
        inputs.append(self.sensor1_cols[iids])
        inputs.append(self.sensor2_cols[iids])
        inputs.append(self.sensor3_cols[iids])
        inputs.append(self.sensor4_cols[iids])
        
        if self.targets is not None: 
            outputs = self.targets[iids]
            return inputs, outputs
        else:
            return inputs

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.idxs)

In [ ]:
import joblib
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import math

In [ ]:
def make_rnn_model(config):
    time_input = tf.keras.layers.Input(shape = (len(TIME_COLUMNS), 1))
    sensor1_input = tf.keras.layers.Input(shape = (len(SENSOR1_COLUMNS), 1))
    sensor2_input = tf.keras.layers.Input(shape = (len(SENSOR2_COLUMNS), 1))
    sensor3_input = tf.keras.layers.Input(shape = (len(SENSOR3_COLUMNS), 1))
    sensor4_input = tf.keras.layers.Input(shape = (len(SENSOR4_COLUMNS), 1))

    decoder_inputs = []
    if config['time_input']:
        decoder_inputs.append(time_input)
    decoder_inputs.append(sensor1_input)
    decoder_inputs.append(sensor2_input)
    decoder_inputs.append(sensor3_input)
    decoder_inputs.append(sensor4_input)

    lstm_inp = tf.keras.layers.Concatenate(axis=-1)(decoder_inputs)               ### shape = (35, 9)  columns = src(1)+dst(1)+zsrc(1)+zeros(1)+len(1)+rho(4)
    raw = lstm_inp
    for i in range(config['rnn_layers']):
        if config['rnn_type'] == 'lstm':
            raw = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(config['lstm_dim'], 
                                    return_sequences=True)
                )(raw)
        else:
            raw = tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(config['lstm_dim'], 
                                    return_sequences=True)
                )(raw) 
        if (i == config['rnn_layers'] - 2) and (config['rnn_dropout'] > 0):
            raw = tf.keras.layers.Dropout(config['rnn_dropout'])(raw) 

    x_rnn1 = tf.keras.layers.GlobalMaxPooling1D()(raw)
    x_rnn2 = tf.keras.layers.GlobalAveragePooling1D()(raw)
    x = tf.keras.layers.Concatenate(axis=-1)([x_rnn1, x_rnn2])
    if config['dense_drop'] > 0:
        x = tf.keras.layers.Dropout(config['dense_drop'])(x)
    x = tf.keras.layers.Dense(config['dense_dim'], activation='relu')(x)
    x = tf.keras.layers.Dense(len(TGT_COLUMNS))(x)
    
    model = tf.keras.Model([time_input, sensor1_input,
                           sensor2_input, sensor3_input, sensor4_input], x)
    
    if config['opt'] == 'adamw':
        opt = tfa.optimizers.AdamW(learning_rate=config['lr'], weight_decay=config['wd'])
    elif config['opt'] == 'sgdw':
        opt = tfa.optimizers.SGDW(learning_rate=config['lr'], 
                                  weight_decay=config['wd'],
                                  momentum=config['momentum'])
    elif config['opt'] == 'radam':
        opt = tfa.optimizers.RectifiedAdam(learning_rate=config['lr'], 
                                           warmup_proportion = 0, 
                                           min_lr = 1e-4,
                                           weight_decay=config['wd'])        
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=config['lr'])
    if config['lookahead']:
        opt = tfa.optimizers.Lookahead(opt, sync_period=config['lk_sync_period'])
    model.compile(loss=config['loss'], optimizer=opt) 
    return model

In [ ]:
class MAEEvaluationSeq(tf.keras.callbacks.Callback):
    def __init__(self, val_gen, ground_truth, trans, name, interval=1):
        super(tf.keras.callbacks.Callback, self).__init__()

        self.X_seq, self.y = val_gen, ground_truth
        self.name = name
        self.interval = interval
        self.trans = trans

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_seq)
            y_pred = self.trans.inverse_transform(y_pred)
            full_mae = mean_absolute_error(self.y, y_pred)
            maes = {}
            for i,f in enumerate(TGT_COLUMNS):
                maes[f] = mean_absolute_error(self.y[:,i], y_pred[:,i])
            maes['full'] = full_mae
            s = 'epoch '+str(epoch)
            #s += f' lr={scheduler(epoch):.2}; '
            for k,d in maes.items():
                s+= ' ' + k + ': ' + f'{d:.4};'
            print(s)
            logs['model_mae'] = full_mae

In [ ]:
def start_experiment(train, target, test, config):
    if config['verbose_exp']:
        print(config)

    kf = KFold(n_splits=config['n_folds'], shuffle=True, random_state=config['seed'])
    fold_metrics = []
    test_preds = np.zeros((test.shape[0], 4))
    train_preds = np.zeros((train.shape[0], 4))
    ifold = 0
    test_gen = DataBatchGenerator(test.copy(), shuffle=False, batch_size=config['batch_size'])
    for tr_ix,va_ix in kf.split(range(train_features.shape[0])):
        x_train = train.loc[tr_ix].reset_index(drop=True)
        y_train = target.loc[tr_ix].reset_index(drop=True)
        x_valid = train.loc[va_ix].reset_index(drop=True)
        y_valid = target.loc[va_ix].reset_index(drop=True)
        
        train_gen = DataBatchGenerator(x_train.copy(), y_train.copy(), shuffle=True, batch_size=config['batch_size'])
        valid_gen = DataBatchGenerator(x_valid.copy(), y_valid.copy(), shuffle=False, batch_size=config['batch_size'])

        def trianfle_fn(x):
            return 1. / (2.**(x - 1))
        clr_f = tfa.optimizers.CyclicalLearningRate(
            initial_learning_rate = config['min_lr'],
            maximal_learning_rate = config['max_lr'],
            step_size = config['cycle_len'],
            scale_fn = trianfle_fn
        )
        clr = tf.keras.callbacks.LearningRateScheduler(clr_f)

        es = tf.keras.callbacks.EarlyStopping(monitor='model_mae', min_delta=0, patience=config['patience'], 
                                          verbose=2, mode='auto', baseline=None, 
                                          restore_best_weights=True)

        ground_truth = target_scaler.inverse_transform(y_valid[TGT_COLUMNS].values)
        met = MAEEvaluationSeq(valid_gen, ground_truth, target_scaler, 'model')

        model = make_rnn_model(config)
        warmup_callbacks, train_callbacks = [], [clr]
        if config['verbose_exp']:
            warmup_callbacks.append(met)
            train_callbacks.append(met)
            if config['early_stop_warmup'] > 0:
                warmup_callbacks.append(es)
        
        model.fit(train_gen, validation_data=valid_gen, 
                    epochs=config['warmup_epochs'], verbose=0, callbacks=warmup_callbacks)
        model.fit(train_gen, validation_data=valid_gen, 
                    epochs=config['train_epochs'], verbose=0, callbacks=train_callbacks) 

        tst_pred = model.predict(test_gen)
        tst_pred = target_scaler.inverse_transform(tst_pred)
        test_preds += tst_pred / config['n_folds']

        predict = model.predict(valid_gen)
        predict = target_scaler.inverse_transform(predict)
        train_preds[va_ix] = predict

        fold_metric = mean_absolute_error(ground_truth, predict)
        print('fold', ifold, 'mae:', fold_metric)
        fold_metrics.append(fold_metric)
        ifold += 1
    return fold_metrics, train_preds, test_preds

In [ ]:
config = {
    'n_folds': 5,
    'seed': 239
}
config['lr'] = 0.003         #warmup lr
config['lstm_dim'] = 80      #dimension of rnn block
config['wd'] = 0.0005        #wright decay for 
config['rnn_type'] = 'gru'   #possible values ['gru','lstm']
config['rnn_layers'] = 6    #number of rnn layers
config['rnn_dropout'] = 0.3
config['dense_dim'] = 128
config['loss'] = 'mae'
config['opt'] = 'adamw'
config['lookahead'] = True
config['lk_sync_period'] = 6
config['dense_drop'] = 0.25
config['momentum'] = 0.9
config['batch_size'] = 200
config['min_lr'] = 0.0001
config['max_lr'] = 0.006
config['cycle_len'] = 20
config['time_input'] = True
config['sensor_input'] = True
config['warmup_epochs'] = 40
config['train_epochs'] = 2*config['cycle_len']+1
config['verbose_exp'] = True
config['early_stop_warmup'] = False
config['patience'] = 0

In [ ]:
fold_metrics, train_preds, test_preds = start_experiment(train_features, train_target, test_features, config)
print('overall:', np.mean(fold_metrics), np.std(fold_metrics))

{'n_folds': 5, 'seed': 239, 'lr': 0.003, 'lstm_dim': 80, 'wd': 0.0005, 'rnn_type': 'gru', 'rnn_layers': 6, 'rnn_dropout': 0.3, 'dense_dim': 128, 'loss': 'mae', 'opt': 'adamw', 'lookahead': True, 'lk_sync_period': 6, 'dense_drop': 0.25, 'momentum': 0.9, 'batch_size': 200, 'min_lr': 0.0001, 'max_lr': 0.006, 'cycle_len': 20, 'time_input': True, 'sensor_input': True, 'warmup_epochs': 40, 'train_epochs': 41, 'verbose_exp': True, 'early_stop_warmup': False, 'patience': 0}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
epoch 0 X: 157.2; Y: 154.7; M: 39.05; V: 0.1616; full: 87.79;
epoch 1 X: 124.0; Y: 102.6; M: 36.2; V: 0.1133; full: 65.72;
epoch 2 X: 86.47; Y: 87.16; M: 34.46; V: 0.1008; full: 52.05;
epoch 3 X: 83.96; Y: 85.6; M: 31.06; V: 0.07126; full: 50.17;
epoch 4 X: 74.53; Y: 79.8; M: 24.69; V: 0.05245; full: 44.77;
epoch 5 X: 82.69; Y: 70.19; M: 18.89; V: 0.04853; full: 42.95;
epoch 6 X: 63.8; Y: 73.62; M: 15.48; V: 0.0462; full: 38.24;
epoch 7 X: 62.3

In [ ]:
sub = pd.read_csv('/content/drive/My Drive/Collider Detection/data/sample_submission.csv')
sub[TGT_COLUMNS] = test_preds
sub.head()

,id,X,Y,M,V
0,2800,-235.644600,-52.434285,109.705200,0.411496
1,2801,315.145916,-304.052975,75.131014,0.427816
2,2802,-237.204247,135.795338,24.763058,0.351630
3,2803,212.809196,286.598122,26.330497,0.365905
4,2804,-132.583000,212.431892,118.215435,0.404628


In [ ]:
sub.to_csv('/content/drive/My Drive/Collider Detection/Dacon_Collider_LSTM_sub.csv', index=False)